In [108]:
!pip install tqdm

In [109]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
from tqdm import tqdm

In [2]:
from bs4 import BeautifulSoup
import requests
url = "https://www.amazon.in/gp/most-gifted/electronics/1389432031/ref=zg_mg_pg_2?ie=UTF8&pg=1"

r = requests.get(url)
r


<Response [200]>

In [3]:
soup = BeautifulSoup(r.content, "lxml")
phones = soup.find_all('a', {'class': 'a-link-normal a-text-normal'})

# links = soup.find_all('a', {'class': 'a-link-normal'})
# reviews = soup.find_all('a', {'class': 'a-link-normal a-text-small'})

phones


[<a class="a-link-normal a-text-normal" href="/Samsung-Galaxy-Ocean-128GB-Storage/dp/B07HGGYWL6?_encoding=UTF8&amp;psc=1"><span class="a-size-base a-color-price"><span class="p13n-sc-price">₹ 17,499.00</span></span></a>,
 <a class="a-link-normal a-text-normal" href="/Test-Exclusive-614/dp/B07HGJJ559?_encoding=UTF8&amp;psc=1"><span class="a-size-base a-color-price"><span class="p13n-sc-price">₹ 13,999.00</span></span></a>,
 <a class="a-link-normal a-text-normal" href="/Redmi-8A-Dual-Blue-Storage/dp/B07X4R63DF?_encoding=UTF8&amp;psc=1"><span class="a-size-base a-color-price"><span class="p13n-sc-price">₹ 7,499.00</span></span></a>,
 <a class="a-link-normal a-text-normal" href="/Test-Exclusive-667/dp/B07HGH8D2R?_encoding=UTF8&amp;psc=1"><span class="a-size-base a-color-price"><span class="p13n-sc-price">₹ 13,999.00</span></span></a>,
 <a class="a-link-normal a-text-normal" href="/Redmi-Note-Neptune-Blue-Storage/dp/B07X1KT6LD?_encoding=UTF8&amp;psc=1"><span class="a-size-base a-color-price

In [4]:
ph_list = []
for phone in phones:
    pl = str(phone.get('href'))
    ph_list.append(pl)
    
len(ph_list)
ph_list

['/Samsung-Galaxy-Ocean-128GB-Storage/dp/B07HGGYWL6?_encoding=UTF8&psc=1',
 '/Test-Exclusive-614/dp/B07HGJJ559?_encoding=UTF8&psc=1',
 '/Redmi-8A-Dual-Blue-Storage/dp/B07X4R63DF?_encoding=UTF8&psc=1',
 '/Test-Exclusive-667/dp/B07HGH8D2R?_encoding=UTF8&psc=1',
 '/Redmi-Note-Neptune-Blue-Storage/dp/B07X1KT6LD?_encoding=UTF8&psc=1',
 '/Samsung-Galaxy-Space-Black-Storage/dp/B07HGN617M?_encoding=UTF8&psc=1',
 '/Test-Exclusive-668/dp/B07HGH88GL?_encoding=UTF8&psc=1',
 '/Samsung-Galaxy-Ocean-Blue-Storage/dp/B07HGJKDQL?_encoding=UTF8&psc=1',
 '/Redmi-Note-Pro-Storage-Processor/dp/B07X4PKGSN?_encoding=UTF8&psc=1',
 '/OnePlus-Nord-Marble-128GB-Storage/dp/B086977J3K?_encoding=UTF8&psc=1',
 '/OnePlus-Nord-Gray-128GB-Storage/dp/B08695ZSP6?_encoding=UTF8&psc=1',
 '/Test-Exclusive-669/dp/B07HGH8ML7?_encoding=UTF8&psc=1',
 '/Redmi-8A-Dual-%E0%A4%A1%E0%A5%8D%E0%A4%AF%E0%A5%81%E0%A4%85%E0%A4%B2/dp/B07X3P1N3H?_encoding=UTF8&psc=1',
 '/Samsung-Galaxy-Space-Black-Storage/dp/B07HGJ7WLM?_encoding=UTF8&psc=1'

In [115]:
def scrape_reviews(link):
    
    link = 'https://www.amazon.in' + link
    page = requests.get(link)
    print("Response = ", page)
    try:
        soup = bs(page.content, 'html.parser')

        header = soup.find_all("h1", class_ = "a-size-large a-spacing-none")
        Ph_name = []
        for i in range(0, len(header)):
            Ph_name.append(header[i].get_text())
        Ph_name[:] = [header.rstrip('\n') for header in Ph_name]
        Ph_name[:] = [header.lstrip('\n') for header in Ph_name]

        names = soup.find_all("span", class_ = "a-profile-name")
        c_name = []
        for i in range(0, len(names)):
            c_name.append(names[i].get_text())

        titles = soup.find_all("a", class_ = "review-title")
        r_title = []
        for j in range(0, len(titles)):
            r_title.append(titles[j].get_text())
        r_title[:] = [titles.rstrip('\n') for titles in r_title]
        r_title[:] = [titles.lstrip('\n') for titles in r_title]

        stars = soup.find_all("i", class_ = "review-rating")
        rating = []
        for j in range(0, len(stars)):
            rating.append(stars[j].get_text())

        review_data = soup.find_all("span", {'data-hook' : 'review-body'})
        reviews = []
        for j in range(0, len(review_data)):
            reviews.append(review_data[j].get_text())
        reviews[:] = [review_data.rstrip('\n') for review_data in reviews]
        reviews[:] = [review_data.lstrip('\n') for review_data in reviews]

        n_p = len(r_title)
        Ph_name = Ph_name * n_p
        
        c_name = c_name[:n_p]
        rating = rating[:n_p]
        reviews = reviews[:n_p]

    except:
        Ph_name, c_name, r_title, rating, reviews = None
    
    print(len(Ph_name), len(c_name), len(r_title), len(rating), len(reviews))
    return  Ph_name, c_name, r_title, rating, reviews


In [116]:
def collect_reviews(ph_list):
    
    df_list = []
    ph_name = []
    c_name = []
    r_title = []
    rating = []
    reviews = []
    
    columns = ['Phone Name', 'Customer Name', 'Review Title', 'Rating', 'Review']
    
    for ph in ph_list:
        result = scrape_reviews(ph)
        ph_name = ph_name + result[0] 
        c_name = c_name + result[1]
        r_title = r_title + result[2]
        rating = rating + result[3]
        reviews = reviews + result[4]

    df = pd.DataFrame(list(zip(ph_name, c_name, r_title, rating, reviews)), columns = columns)
    return  df

In [117]:
phone_data = collect_reviews(ph_list)
phone_data

Response =  <Response [503]>
0 0 0 0 0
Response =  <Response [503]>
0 0 0 0 0
Response =  <Response [503]>
0 0 0 0 0
Response =  <Response [503]>
0 0 0 0 0
Response =  <Response [503]>
0 0 0 0 0
Response =  <Response [503]>
0 0 0 0 0
Response =  <Response [503]>
0 0 0 0 0
Response =  <Response [503]>
0 0 0 0 0
Response =  <Response [503]>
0 0 0 0 0
Response =  <Response [503]>
0 0 0 0 0
Response =  <Response [503]>
0 0 0 0 0
Response =  <Response [503]>
0 0 0 0 0
Response =  <Response [503]>
0 0 0 0 0
Response =  <Response [503]>
0 0 0 0 0
Response =  <Response [503]>
0 0 0 0 0
Response =  <Response [503]>
0 0 0 0 0
Response =  <Response [200]>
8 8 8 8 8
Response =  <Response [503]>
0 0 0 0 0
Response =  <Response [503]>
0 0 0 0 0
Response =  <Response [503]>
0 0 0 0 0
Response =  <Response [503]>
0 0 0 0 0
Response =  <Response [503]>
0 0 0 0 0
Response =  <Response [200]>
8 8 8 8 8
Response =  <Response [503]>
0 0 0 0 0
Response =  <Response [503]>
0 0 0 0 0
Response =  <Response [50

,Phone Name,Customer Name,Review Title,Rating,Review
0,"OPPO A5 2020 (Dazzling White, 4GB RAM, 64GB St...",sanket yavalkar,Good opportunity for oppo lovers to upgrade,5.0 out of 5 stars,Brought this phone for my wife who is die ha...
1,"OPPO A5 2020 (Dazzling White, 4GB RAM, 64GB St...",Pj,Best budget phone with all features,5.0 out of 5 stars,"Best option for exchange, I got it in 9k ,Re..."
2,"OPPO A5 2020 (Dazzling White, 4GB RAM, 64GB St...",Amit,Good,5.0 out of 5 stars,Good Phone With Exchange\n\nRead more
3,"OPPO A5 2020 (Dazzling White, 4GB RAM, 64GB St...",Javed Shaikh,Come and collect,1.0 out of 5 stars,Lowest speed ever seen in android and charge...
4,"OPPO A5 2020 (Dazzling White, 4GB RAM, 64GB St...",siddheshpatil,Good Phone at this price range..by Oppo,4.0 out of 5 stars,Oppo makes very nice phone at this price ran...
5,"OPPO A5 2020 (Dazzling White, 4GB RAM, 64GB St...",Brijesh Yadav,Very nice,4.0 out of 5 stars,Nyc\n\nRead more
6,"OPPO A5 2020 (Dazzling White, 4GB RAM, 64GB St...",Vishal kumar,Slow charging and hand free is not available.,1.0 out of 5 stars,The first thing is slow charging of this mob...
7,"OPPO A5 2020 (Dazzling White, 4GB RAM, 64GB St...",Tabassum,Not recommended,2.0 out of 5 stars,Pros- Phone looks actually good Good batter...
8,"OnePlus 7T (Glacier Blue, 8GB RAM, Fluid AMOLE...",Praggya,Think and evaluate before you go for it,3.0 out of 5 stars,After reading a lot of geeky reviews on YouT...
9,"OnePlus 7T (Glacier Blue, 8GB RAM, Fluid AMOLE...",Amazon Customer,Treat to the eye,4.0 out of 5 stars,Detailed review --Design - 4/5+Reduced notch...


In [118]:
phone_data

,Phone Name,Customer Name,Review Title,Rating,Review
0,"OPPO A5 2020 (Dazzling White, 4GB RAM, 64GB St...",sanket yavalkar,Good opportunity for oppo lovers to upgrade,5.0 out of 5 stars,Brought this phone for my wife who is die ha...
1,"OPPO A5 2020 (Dazzling White, 4GB RAM, 64GB St...",Pj,Best budget phone with all features,5.0 out of 5 stars,"Best option for exchange, I got it in 9k ,Re..."
2,"OPPO A5 2020 (Dazzling White, 4GB RAM, 64GB St...",Amit,Good,5.0 out of 5 stars,Good Phone With Exchange\n\nRead more
3,"OPPO A5 2020 (Dazzling White, 4GB RAM, 64GB St...",Javed Shaikh,Come and collect,1.0 out of 5 stars,Lowest speed ever seen in android and charge...
4,"OPPO A5 2020 (Dazzling White, 4GB RAM, 64GB St...",siddheshpatil,Good Phone at this price range..by Oppo,4.0 out of 5 stars,Oppo makes very nice phone at this price ran...
5,"OPPO A5 2020 (Dazzling White, 4GB RAM, 64GB St...",Brijesh Yadav,Very nice,4.0 out of 5 stars,Nyc\n\nRead more
6,"OPPO A5 2020 (Dazzling White, 4GB RAM, 64GB St...",Vishal kumar,Slow charging and hand free is not available.,1.0 out of 5 stars,The first thing is slow charging of this mob...
7,"OPPO A5 2020 (Dazzling White, 4GB RAM, 64GB St...",Tabassum,Not recommended,2.0 out of 5 stars,Pros- Phone looks actually good Good batter...
8,"OnePlus 7T (Glacier Blue, 8GB RAM, Fluid AMOLE...",Praggya,Think and evaluate before you go for it,3.0 out of 5 stars,After reading a lot of geeky reviews on YouT...
9,"OnePlus 7T (Glacier Blue, 8GB RAM, Fluid AMOLE...",Amazon Customer,Treat to the eye,4.0 out of 5 stars,Detailed review --Design - 4/5+Reduced notch...


In [557]:
# a = 'abcd'
# lx = []
# lx.append(a)

# lx = lx*5

# lx

In [559]:
# link = 'https://www.amazon.in/Redmi-Note-Moonlight-White-Storage/dp/B07X1KT1FJ?_encoding=UTF8&psc=1'

# r = requests.get(link)
# print(r)
# soup = BeautifulSoup(r.content, "lxml")

# titles = soup.find_all("a", class_ = "review-title")
# titles1 = soup.find_all("span", class_ = "review-title")
# r_title = []
# for j in range(0, len(titles)):
#     print(titles[j].get_text())

# #     r_title.append(titles[j].get_text())


# r_title[:] = [titles.rstrip('\n') for titles in r_title]
# r_title[:] = [titles.lstrip('\n') for titles in r_title]



# for j in range(0, len(titles1)):
#     print(titles1[j].get_text())


In [57]:
# import requests
# from bs4 import BeautifulSoup as bs

# r = requests.get('https://www.amazon.in/gp/most-gifted/electronics/1389432031/ref=zg_mg_pg_2?ie=UTF8&pg=1')
# soup = bs(r.content, 'lxml')
# items = [item['alt'] for item in soup.select('.a-link-normal [alt]')]

# for x in items:
#     df = pd.DataFrame(data_json['query']['results']['quote'])
#     df_list.append(df)

# quote_df = pd.concat(df_list)
# quote_df.to_csv('stocks.csv')
# print(items)

['Samsung Galaxy M31 (Ocean Blue, 6GB RAM, 128GB Storage)', 'Samsung Galaxy M21 (Midnight Blue, 4GB RAM, 64GB Storage)', 'Redmi 8A Dual (Sea Blue, 2GB RAM, 32GB Storage) – Dual Cameras & 5,000 mAH Battery', 'Samsung Galaxy M21 (Raven Black, 4GB RAM, 64GB Storage)', 'Redmi Note 8 (Neptune Blue, 4GB RAM, 64GB Storage)', 'Samsung Galaxy M31 (Space Black, 6GB RAM, 128GB Storage)', 'Redmi 8A Dual (Sea Blue, 3GB RAM, 32GB Storage) – Dual Cameras & 5,000 mAH Battery', 'Samsung Galaxy M21 (Midnight Blue, 6GB RAM, 128GB Storage)', 'Samsung Galaxy M31 (Ocean Blue, 6GB RAM, 64GB Storage)', 'Redmi Note 8 (Moonlight White, 4GB RAM, 64GB Storage)', 'Redmi Note 8 Pro (Halo White, 6GB RAM, 128GB Storage with Helio G90T Processor)', 'OnePlus Nord 5G (Blue Marble, 8GB RAM, 128GB Storage)', 'OnePlus Nord 5G (Gray Onyx, 8GB RAM, 128GB Storage)', 'Redmi 8A Dual (Sea Blue, 3GB RAM, 64GB Storage) – Dual Cameras & 5,000 mAH Battery', 'OnePlus Nord 5G (Gray Onyx, 12GB RAM, 256GB Storage)', 'Samsung Galaxy M21 